In [1]:
import pandas as pd
import requests
import os
import openai
import o4c

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Leer la llave para el API de OpenAI
with open('/Users/ken/.openai.key') as f:
    openai.api_key  = f.read().rstrip() 

## 1. Obtener interpretaciones, sus resúmenes y sus embeddings

In [4]:
interpretaciones_raw = 'Interpretaciones O4C (Respuestas) - Respuestas de formulario 1.csv'
interpretaciones_embed = 'interpretaciones_embeddings.csv'
interpretaciones_resumidas='interpretaciones_resumidas.csv'

# Si el archivo "crudo" es más reciente, reprocesar
if os.path.getmtime(interpretaciones_embed) > os.path.getmtime(interpretaciones_raw):
    if os.path.getmtime(interpretaciones_resumidas) > os.path.getmtime(interpretaciones_embed):
        interpretaciones = pd.read_csv(interpretaciones_resumidas)
        import ast
        interpretaciones['embeddings']=interpretaciones.embeddings.apply(
            lambda s: list(ast.literal_eval(s)))
    else:
        # Leer archivo con embeddings
        interpretaciones = pd.read_csv(interpretaciones_embed)
        import ast
        interpretaciones['embeddings']=interpretaciones.embeddings.apply(
            lambda s: list(ast.literal_eval(s)))
        # Resumir 
        interpretaciones = o4c.get_interpretaciones_resumidas(interpretaciones,max_char = 800)
        interpretaciones.to_csv(interpretaciones_resumidas)
else:
    # Leer el archivo descargado de google
    interpretaciones = o4c.leer_interpretaciones(interpretaciones_raw)
    # Obtener las referencias de crossref
    interpretaciones[['Referencia','Año','Título']] = interpretaciones['DOI'].apply( 
        lambda x: pd.Series(o4c.get_reference(x))) 
    # Calcular los embeddings usando ADA 0002
    interpretaciones = o4c.get_embed_interpretaciones(interpretaciones)
    # Resumir 
    interpretaciones = o4c.get_interpretaciones_resumidas(interpretaciones,max_char = 800)
    interpretaciones.to_csv(interpretaciones_resumidas)